Let's define a function that takes the state of the system $s = (\vec \rho(T), \vec v(T))$ and returns $(\dot {\vec \rho}(T) , \dot {\vec v}(T))$

In [1]:
from numpy import sqrt, abs
import numpy as np

# Funktion som finder længden af rho givet x,y
def rhoFind(x,y):
    rho = sqrt(x**2 + y**2)
    return rho

# Funktion som finder lambda givet x,y,vx,vy
def lamFind(x,y,vx,vy):
    lam = abs(vx*y - vy*x)
    return lam

# Funktion som tager startbetingelserne (s = [rhox, rhoy, vx, vy]) og beregner de tidsafledte bevægelsesligninger (rhomax, og RS er inkluderet pga 'events' i solve_ivp funktionen)
def CLdiffs(t, s, rhomax, RS):
    x,y,vx,vy=s
    rho = rhoFind(x,y)
    K = 1
    
    dx = vx
    dy = vy
    dvx = -x/(2*rho**3)*K
    dvy = -y/(2*rho**3)*K
    return [dx, dy, dvx, dvy]

# Funktion som tager startbetingelserne (s = [rhox, rhoy, vx, vy]) og beregner de tidsafledte bevægelsesligninger (rhomax, og RS er inkluderet pga 'events' i solve_ivp funktionen)
def GRdiffs(t, s, rhomax, RS):
    x,y,vx,vy=s
    rho = rhoFind(x,y)
    lam = lamFind(x,y,vx,vy)
    K = 1 + (3*lam**2)/rho**2
    
    dx = vx
    dy = vy
    dvx = -x/(2*rho**3)*K
    dvy = -y/(2*rho**3)*K
    return [dx, dy, dvx, dvy]

Now we define a function that takes that state of the system $s$ and returns the distance of $m$ to $R_S$ og $2 \left\lVert \vec \rho_0 \right\rVert$ respectively.

In [2]:
# En funktion der returnerer forskellen mellem rho og RS givet en tid og en systemtilstand.
def dyk(t, s, rhomax, RS = 1):
    x, y, vx, vy=s
    rho = rhoFind(x,y)
    return rho - RS
dyk.terminal = True

# En funktion der returnerer forskellen på rhomax og rho givet en tid og en systemtilstand.
def sprædning(t, s, rhomax, RS = 1):
    x,y,vx,vy=s
    rho = rhoFind(x,y)
    return rhomax - rho
sprædning.terminal = True

Now we define a function that solves the set of coupled differential equations. (More info in this markdown cell)

<!-- Nu skriver vi en funktion som sammenfatter de sidste par funktioner, til en numerisk løsning af vores begyndelsesværdiproblem ved brug af *scipy.integrate.solve_ivp*-funktionen. Funktionen tager som udgangspunkt systemets begyndelsestilstand $s = [\overrightarrow{\rho_0}, \overrightarrow{v_0}]$. Evt. kan den også tage en største integrationsstid (grænsen for de bundne orbitaler) som er sat til 2000 som standard, en $R_S$-værdi som er sat til 1 og en faktor med hvilken man kan skalere grænsen for sprædningsorbitalerne, som standard er den sat til 2. Funktionen returnerer så alle tiderne $ T = \left\{T_i \right\}$, positionerne $R = \left\{\overrightarrow{\rho}(T_i) | T_i \in T \right\}$, hastighederne $Y = \left\{\overrightarrow{v}(T)| T_i \in T \right\}$ og afstandende $D = \left\{||\overrightarrow{\rho}(T_i)|| \,| \,T_i \in T \right\}$. Yderligere returnerer funktionen også til hvilken tid en af grænserne (radiel dyk eller sprædningsorbital) blev nået. Hvis ikke nogen grænse blev nået returnerer den bare $T_{max}$. -->

In [3]:
from scipy.integrate import solve_ivp
from numpy import linspace

# Funtkion som returnerer tider, positioner, hastigheder, afstande og integrationstid for orbitaler, givet startværdier og evt en øvre tidsgrænse
def orbitals(s, Tmax = 2000, RS = 1, d=2, GR = False, resolution = 20000):
    y0 = x,y,vx,vy = s
    rhomax = d*rhoFind(s[0],s[1])
    t_span = [0,Tmax]
    t_eval = linspace(t_span[0], t_span[1], resolution)
    
    if GR == False:
        sol = solve_ivp(CLdiffs, t_span = t_span, y0 = y0, t_eval = t_eval, events = [sprædning, dyk], method = 'DOP853', args=(rhomax, RS))
    else:
        sol = solve_ivp(GRdiffs, t_span = t_span, y0 = y0, t_eval = t_eval, events = [sprædning, dyk], method = 'DOP853', args=(rhomax, RS))
    tim = sol.t
    pos = [sol.y[0], sol.y[1]]
    vel = [sol.y[2], sol.y[3]]
    t_events = sol.t_events
    
    rho = rhoFind(pos[0],pos[1])
    
    if len(t_events[0]) > 0:
        Tmax = t_events[0][0]
    elif len(t_events[1]) > 0:
        Tmax = t_events[1][0]
    else:
        Tmax = Tmax
    return tim, pos, vel, rho, Tmax, sol

And a function that can plot the orbitals

In [4]:
import matplotlib.pyplot as plt

def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

# Funktion der plotter en orbital med start- og slutposition og RS givet en liste med positioner
def orbPlotter(pos, filename, RS = 1, GR = False, save = False, show = True):
    start_pos = [pos[0][0], pos[1][0]]
    end_pos = [pos[0][-1], pos[1][-1]]
    
    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    if GR == True:
        body = plt.Circle(xy=(0, 0), radius=RS, color='k')
            
        fig, ax = plt.subplots(figsize=(11,11))
        
        ax.set_xlim(xmin-0.2*max_pos, xmax+0.2*max_pos)
        ax.set_ylim(ymin-0.2*max_pos, ymax+0.2*max_pos)

        ax.plot(pos[0],pos[1], 'k', label = 'Orbital', linewidth=0.75)
        ax.plot(start_pos[0], start_pos[1], 'kx', label = f'Start pos: ({"%.1f" % start_pos[0]}, {"%.1f" % start_pos[1]})')
        ax.plot(end_pos[0],end_pos[1], 'k+', label = f'End pos: ({"%.1f" % end_pos[0]}, {"%.1f" % end_pos[1]})')
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
        # ax.set_facecolor('k')
        ax.grid(c='grey', alpha=0.2, ls ='--')
        ax.set_aspect('equal')
        ax.add_artist(body)
        ax.set_title(f'{filename}', fontsize = 20)
        ax.legend(facecolor='grey',loc='upper right', fontsize = 12)

    else:
        body = plt.Circle(xy=(0, 0), radius=RS, color='w')
        
        fig, ax = plt.subplots(figsize=(11,11))
        
        ax.set_xlim(xmin-0.2*max_pos, xmax+0.2*max_pos)
        ax.set_ylim(ymin-0.2*max_pos, ymax+0.2*max_pos)

        ax.plot(pos[0],pos[1], 'w', label = 'Orbital',linewidth=0.75)
        ax.plot(start_pos[0], start_pos[1], 'wx', label = f'Start pos: ({"%.1f" % start_pos[0]}, {"%.1f" % start_pos[1]})')
        ax.plot(end_pos[0],end_pos[1], 'w+', label = f'End pos: ({"%.1f" % end_pos[0]}, {"%.1f" % end_pos[1]})')
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
        ax.set_facecolor('k')
        ax.grid(c='grey', alpha=0.2, ls ='--')
        ax.set_aspect('equal')
        ax.add_artist(body)
        ax.set_title(f'{filename}', fontsize = 20)
        ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300)

And a function that plots the orbits in a prettier way

In [5]:
import matplotlib.pyplot as plt

# Funktion der plotter en orbital givet en liste med positioner
def prettyOrbPlotter(pos, filename, RS = 1, GR = False, save = False, show = True):
    start_pos = [pos[0][0], pos[1][0]]
    end_pos = [pos[0][-1], pos[1][-1]]
    
    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    if GR == True:
        body = plt.Circle(xy=(0, 0), radius=RS, color='k')
            
        fig, ax = plt.subplots(figsize=(11,11))
        
        ax.set_xlim(xmin-0.2*max_pos, xmax+0.2*max_pos)
        ax.set_ylim(ymin-0.2*max_pos, ymax+0.2*max_pos)

        ax.plot(pos[0],pos[1], 'k', label = 'Orbital', linewidth=0.75)
        ax.plot(start_pos[0], start_pos[1], 'kx', label = f'Start pos: ({"%.1f" % start_pos[0]}, {"%.1f" % start_pos[1]})')
        ax.plot(end_pos[0],end_pos[1], 'k+', label = f'End pos: ({"%.1f" % end_pos[0]}, {"%.1f" % end_pos[1]})')
        # ax.set_xlabel('$R_S$', fontsize = 15)
        # ax.set_ylabel('$R_S$', fontsize = 15)
        # ax.set_facecolor('k')
        # ax.grid(c='grey', alpha=0.2, ls ='--')
        ax.set_aspect('equal')
        ax.add_artist(body)
        # ax.set_title(f'{filename}', fontsize = 20)
        # ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
        plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)

    else:
        body = plt.Circle(xy=(0, 0), radius=RS, color='w')
        
        fig, ax = plt.subplots(figsize=(11,11))
        
        ax.set_xlim(xmin-0.2*max_pos, xmax+0.2*max_pos)
        ax.set_ylim(ymin-0.2*max_pos, ymax+0.2*max_pos)

        ax.plot(pos[0],pos[1], 'w', label = 'Orbital',linewidth=0.5)
        ax.plot(start_pos[0], start_pos[1], 'wx', label = f'Start pos: ({"%.1f" % start_pos[0]}, {"%.1f" % start_pos[1]})')
        ax.plot(end_pos[0],end_pos[1], 'wx', label = f'End pos: ({"%.1f" % end_pos[0]}, {"%.1f" % end_pos[1]})')
        # ax.set_xlabel('$R_S$', fontsize = 15)
        # ax.set_ylabel('$R_S$', fontsize = 15)
        ax.set_facecolor('k')
        # ax.grid(c='grey', alpha=0.2, ls ='--')
        ax.set_aspect('equal')
        ax.add_artist(body)
        # ax.set_title(f'{filename}', fontsize = 20)
        # ax.legend(facecolor='grey',loc='upper right', fontsize = 12)
        plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/Pretty/{filename}.png', dpi=300)


Here we're defining a function that finds the momentum $P$ and $P_\infty$ given an energy $E$ 

In [7]:
def findP(E, α, β, κ=1):
    ρ = sqrt(α**2 + β**2)
    P = sqrt(2*(E + κ/ρ)/(1 + (β/ρ)**2))
    return P

def findPinf(E, α, β, κ=1):
    ρ = sqrt(α**2 + β**2)
    Pinf = sqrt(2*E)
    return Pinf